In [1]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="mesolitica/Malaysian-UltraChat-Speech-Multiturn-Instructions",
    repo_type='dataset',
    allow_patterns="data/*.parquet",
    local_dir="./Malaysian-UltraChat-Speech-Multiturn-Instructions",
)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

'/home/mesolitica/stt/Malaysian-UltraChat-Speech-Multiturn-Instructions'

In [7]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import os
import json

In [12]:
files = glob('Malaysian-UltraChat-Speech-Multiturn-Instructions/data/*.parquet')

In [5]:
summary = pd.read_parquet('summarization.parquet')
summary.head()

,answer,text
0,Malaysia's public transport system is fragment...,Sistem pengangkutan awam Malaysia telah mengha...
1,Malaysia is vulnerable to climate change due t...,Malaysia is indeed one of the countries most v...
2,PDRM berusaha keras untuk menangani aktiviti A...,Perkembangan terkini dalam menangani isu aktiv...
3,Usaha membantu masyarakat Bajau di Sabah terma...,"Untuk membantu masyarakat Bajau di Sabah, terd..."
4,Saya adalah AI yang direka untuk membantu dan ...,"Itulah betul! Saya adalah program komputer, te..."


In [8]:
summaries = {}
for i in tqdm(range(len(summary))):
    summaries[summary['text'].iloc[i]] = summary['answer'].iloc[i]

100%|██████████████████████████████████████████████████████████████████████████| 1264550/1264550 [00:15<00:00, 81973.56it/s]


In [9]:
system_prompt = [
    'you are chatbot that use for voice assistant, your response always precise and shorter than 300 characters',
    'You are a chatbot designed specifically for use as a voice assistant. Your responses must be clear, precise, and always shorter than 300 characters. Prioritize natural, conversational language suitable for speech. Avoid filler, long explanations, or complex phrasing. Do not include unnecessary details unless explicitly asked. If a user’s request is ambiguous, briefly ask for clarification within the character limit. Aim to be helpful, fast, and concise in all replies.',
    'You are a helpful voice assistant chatbot. Always reply with short, clear answers under 300 characters. Speak naturally, like in conversation. Keep it direct—no fluff, no long explanations. If you’re unsure, ask a quick clarifying question.',
    'Act as a voice assistant chatbot. Keep every response under 300 characters. Be accurate, brief, and easy to understand when spoken aloud. Don’t overexplain or repeat. Ask for clarification only when needed. Prioritize clarity and brevity at all times.',
    "You're a chatbot for voice use. Talk like a helpful person, not a machine. Keep it short—under 300 characters. Focus on what the user needs. Don’t ramble. Be polite, fast, and smart. If the user’s unclear, just ask quickly and keep going.",
    'You are a voice-based assistant designed to be brief and precise. All replies must be under 300 characters. Speak in a friendly, natural tone suitable for spoken output. Avoid technical jargon and long replies. Ask short questions if more info is needed.',
]

In [14]:
rows = []
for f in files:
    df = pd.read_parquet(f).to_dict(orient = 'records')
    rows.extend(df)

In [25]:
rows[0].keys()

dict_keys(['conversation', 'voice', 'audio_filenames'])

In [28]:
import random

data = []
for i in tqdm(range(len(rows))):
    conversation = json.loads(rows[i]['conversation'])
    messages = [{'role': 'system', 'content': random.choice(system_prompt)}]
    for c in conversation:
        if c['role'] == 'assistant':
            s = c['content']
            if len(c['content']) > 300:
                if c['content'] in summaries:
                    s = summaries[c['content']]
                    if s is None:
                        s = c['content']
                else:
                    break
            c['content'] = s
        messages.append(c)
        
    if len(messages) % 2 == 0:
        continue
        
    data.append({
        'conversation': json.dumps(messages),
        'voice': rows[i]['voice'],
        'audio_filenames': rows[i]['audio_filenames'],
    })

100%|██████████████████████████████████████████████████████████████████████████████| 97692/97692 [00:03<00:00, 25097.83it/s]


In [29]:
len(data)

95129

In [30]:
from datasets import Dataset

dataset = Dataset.from_list(data)

In [ ]:
dataset.push_to_hub('mesolitica/Malaysian-UltraChat-Speech-Multiturn-Instructions', split = 'voice_assistant')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/96 [00:00<?, ?ba/s]